### PACKAGE

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib import colors as mcolors
import os
import package_DBR
import package_lab

from importlib import reload
package_DBR = reload(package_DBR)
package_lab = reload(package_lab)

from package_DBR import *
from package_lab import *

### Simulation parameters

In [6]:
#SOPDT from PV
Kp = 0.398981565560627
T1p = 165.56685644081972
T2p = 3.645364787323084e-05
thetap = 0.6102455489040157

# from DV
Kd = 0.31148119513939204
T1d = 167.0472401651847
T2d = 14.406792795634894
thetad = 9.375122302137418

# Define system parameters
TSim = 2000
Ts = 1
N = int(np.ceil(TSim / Ts)) + 1

MVPath = {0: 1, 1230: 0, TSim: 1}

# Define operating point
MV0 = 50
DV0 = 50
PV0 = 75

# Create a dictionary of parameters for the process
process_parameters = {
    "Kp": Kp,
    "Tlead1": 0,  # Assuming no lead
    "Tlag1": T1p,
    "Tlag2": T2p,
    "theta": thetap,
    "nInt": 0  # Assuming no additional integrator
}

# Initialize the Process object with the parameters dictionary
P = Process(process_parameters)

# Define initial controller parameters (placeholders)
Kc = 1.0
alpha = 1
Ti = 0.0
Td = 0.0

# Create a dictionary of parameters for the PID controller
pid_parameters = {
    'Kc': Kc,
    'alpha': alpha,
    'Ti': Ti,
    'Td': Td
}

# Initialize the PID controller object
C = PID(pid_parameters)

# Tuning the controller
gamma = 0.6
Kc,Ti,Td = IMCTuning(Kp,T1p,T2p,thetap,gamma,'SOPDT')

# Update controller parameters after tuning
Kc = C.parameters["Kc"]
Ti = C.parameters["Ti"]
Td = C.parameters["Td"]

# Print the tuned controller parameters
print('Tuned Controller Parameters:')
print('Kc:', Kc)
print('Ti:', Ti)
print('Td:', Td)


Kp : 0.398981565560627, Tlag1 : 165.56685644081972, Tlag2 : 3.645364787323084e-05, thetha : 0.6102455489040157, Kc : 4.151798889695989, Ti : 165.5668928944676, Td : 3.645363984705833e-05
Tuned Controller Parameters:
Kc: 1.0
Ti: 0.0
Td: 0.0


### Simulation Input Signals

In [7]:
scenario_choice = 2  # Change this to 1, 2, 3, or 4 to select a scenario

if scenario_choice == 1:
    # Scenario 1: OLP + no FF
    ManPath = {0: True, 'TSim': True}
    MVManPath = {0: MV0, 'TSim': MV0} 
    SPPath = {0: PV0, 'TSim': PV0} 
    DVPath = {0: 50.0, 1000: 60, 'TSim': 60.0} 
    ActivateFF = False
    ManFF = False

elif scenario_choice == 2:
    # Scenario 2: OLP + FF
    ManPath = {0: True, 'TSim': True} 
    MVManPath = {0: MV0, 'TSim': MV0} 
    SPPath = {0: PV0, 'TSim': PV0} 
    DVPath = {0: 50.0, 1000: 60, 'TSim': 60.0} 
    ActivateFF = True
    ManFF = True

elif scenario_choice == 3:
    # Scenario 3: CLP + no FF
    ManPath = {0: True, 500: False, 'TSim': False} 
    MVManPath = {0: MV0+10, 'TSim': MV0+10} 
    SPPath = {0: PV0+5, 1000: PV0-5, 'TSim': PV0-5} 
    DVPath = {0: DV0, 1500: DV0+10, 'TSim': DV0+10} 
    ActivateFF = False
    ManFF = False

elif scenario_choice == 4:
    # Scenario 4: CLP + FF
    ManPath = {0: True, 500: False, 'TSim': False} 
    MVManPath = {0: MV0+10, 'TSim': MV0+10} 
    SPPath = {0: PV0+5, 1000: PV0-5, 'TSim': PV0-5} 
    DVPath = {0: DV0, 1500: DV0+10, 'TSim': DV0+10} 
    ActivateFF = True
    ManFF = False

else:
    print("Invalid scenario choice")



### Closed-loop simulation with PID and FF

In [8]:
#running simulation

#initialize arrays of data
t = []

SP = []
PV = []
MAN = []
MV_MAN = []
FF = []
PV_P = []
DV = []
PV_D = []
MV_FF = []

MV = []
MVp = []
MVi = []
MVd = []
E = []

#intermediate MV values
MV_P_delay = []
MV_P_FO_delay = []
MVFF_delay = []
MVFF_LL = []
MV_D_delay = []
MV_D_FO_delay = []

# Ensure SPPath is a dictionary before converting its keys to integers
if isinstance(SPPath, dict):
    # Remove non-numeric keys from SPPath dictionary
    SPPath = {int(key): value for key, value in SPPath.items() if str(key).isdigit()}

    # Ensure ManPath is a dictionary before converting its keys to integers
    if isinstance(ManPath, dict):
        # Remove non-numeric keys from ManPath dictionary
        ManPath = {int(key): value for key, value in ManPath.items() if str(key).isdigit()}

        # Now, SPPath and ManPath only contain keys that are integers, and you can pass them to SelectPath_RT function
        for i in range(0, N):
            t.append(i * Ts)
            # Convert time values to integers
            int_t = [int(time_val) for time_val in t]
            SelectPath_RT(SPPath, int_t, SP)
            SelectPath_RT(ManPath, int_t, MAN)
            SelectPath_RT(MVManPath, int_t, MV_MAN)
            SelectPath_RT(FFPath, int_t, FF)
            SelectPath_RT(DVPATh, int_t, DV)



            # Feed Forward
            Delay_RT(DV - DV0 * np.ones_like(DV), np.max([thetad - thetap, 0]), Ts, MVFF_delay)
            LL_RT(MVFF_delay, -Kd / Kp, T1p, T1d, Ts, MVFF_LL)
            LL_RT(MVFF_LL, 1, T2p, T2d, Ts, MV_FF)

            # PID
            PID_RT(SP, PV, Man, MV_MAN, FF, MV_FF, PVInit, Kc, Ti, Td, alpha, MVmin, MVmax, Ts, MV, MVp, MVi, MVd, E)

            # Process
            Delay_RT(MV, thetap, Ts, MV_P_delay, MV0)
            FO_RT(MV_P_delay, Kp, T1p, Ts, MV_P_FO_delay, 0)
            FO_RT(MV_P_FO_delay, 1, T2p, Ts, PV_P, 0)

            # Disturbance
            Delay_RT(DV - DV0 * np.ones_like(DV), thetad, Ts, MV_D_delay, 0)
            FO_RT(MV_D_delay, Kd, T1d, Ts, MV_D_FO_delay, 0)
            FO_RT(MV_D_FO_delay, 1, T2d, Ts, PV_D, 0)

            PV.append(PV_P[-1] + PV_D[-1] + PV0 - Kp * MV0)
    else:
        print("ManPath is not a dictionary. Please ensure that ManPath is initialized as a dictionary.")
else:
    print("SPPath is not a dictionary. Please ensure that SPPath is initialized as a dictionary.")


NameError: name 'Man' is not defined

### Plot

In [ ]:
fig, (ax1,ax2,ax3,ax4) = plt.subplots(4,1)
fig.set_figheight(22)
fig.set_figwidth(22)

l1, = ax1.step([0,t[-1]],[0,100],'k-',linewidth=2,label='Man',where='post')
ax1.set_ylabel('Man [0 or 1]')
ax1.set_title(Title_loop + ' response with PID controller and ' + Title_FF)
ax1.legend(loc='best')

l2, = ax2.step([0,t[-1]],[0,100],'b-',linewidth=2,label='MV',where='post')
ax2.set_ylabel('MV [%]')
ax2.legend(loc='best')

l3, = ax3.step([0,t[-1]],[0,100],'k-',linewidth=2,label='SP',where='post')

l4, = ax3.step([0,t[-1]],[0,100],'g-',linewidth=2,label='PV',where='post')
ax3.set_ylabel('PV [°C]')
ax3.legend(loc='best')

l5, = ax4.step([0,t[-1]],[0,100],'r-',linewidth=2,label='DV',where='post')
ax4.set_xlabel('Time [s]')
ax4.set_ylabel('DV [%]')
ax4.legend(loc='best')

ManInt = [int(x) for x in Man]
l1.set_data(t,ManInt)
l2.set_data(t,MV)
l3.set_data(t,SP)
l4.set_data(t,PV)
l5.set_data(t,DV)

ax1.set_xlim(0,t[-1]+1)
ax2.set_xlim(0,t[-1]+1)
ax3.set_xlim(0,t[-1]+1)
ax4.set_xlim(0,t[-1]+1)

ax1.set_ylim(-0.1,1.1)
ax2.set_ylim(myRound(np.min(MV),5)-5, myRound(np.max(MV),5)+5)
ax3.set_ylim(myRound(np.min((np.min(PV),np.min(SP))),5)-5,myRound(np.max((np.max(PV),np.max(SP))),5)+5)
ax4.set_ylim(myRound(np.min(DV),5)-5,myRound(np.max(DV),5)+5)

### Comparing response speed with different gamma values


In [ ]:
#running simulation

#initialize arrays of data
t = []

SP = []
PV_low_gamma = []
MAN = []
MV_MAN = []
FF = []
PV_P = []
DV = []
PV_D = []
MV_FF = []

MV_low_gamma = []
MVp = []
MVi = []
MVd = []
E = []

#intermediate MV values
MV_P_delay = []
MV_P_FO_delay = []
MVFF_delay = []
MVFF_LL = []
MV_D_delay = []
MV_D_FO_delay = []

for i in range(0,N):
    t.append(i*Ts)
    SelectPath_RT(SPPath,t,SP)
    SelectPath_RT(ManPath,t,MAN)
    SelectPath_RT(MVmanPath,t,MV_MAN) 
    SelectPath_RT(FFPath,t,FF)
    SelectPath_RT(DVPATh,t,DV)
    
    #Feed Forward
    Delay_RT(DV-DV0*np.ones_like(DV), np.max([thetad-thetap,0]), Ts, MVFF_delay)
    LL_RT(MVFF_delay,-Kd/Kp,T1p,T1d,Ts,MVFF_LL)
    LL_RT(MVFF_LL,1,T2p,T2d,Ts,MV_FF)
    
    #PID
    PID_RT(SP, PV_low_gamma, MAN, MV_MAN, FF, MV_FF, PVInit, Kc_low_gamma, Ti_low_gamma, Td_low_gamma, alpha, MVmin, MVmax, Ts, MV_low_gamma, MVp, MVi, MVd, E)
    
    #Process
    Delay_RT(MV_low_gamma,thetap,Ts, MV_P_delay, MV0) 
    FO_RT(MV_P_delay, Kp, T1p, Ts, MV_P_FO_delay, 0)
    FO_RT(MV_P_FO_delay, 1, T2p, Ts, PV_P, 0)
        
    #Disturbance
    Delay_RT(DV-DV0*np.ones_like(DV), thetad, Ts, MV_D_delay, 0)
    FO_RT(MV_D_delay, Kd, T1d, Ts, MV_D_FO_delay, 0)
    FO_RT(MV_D_FO_delay, 1, T2d, Ts,PV_D, 0)
    
    PV_low_gamma.append(PV_P[-1] + PV_D[-1] + PV0 - Kp * MV0)

plt.figure(figsize = (15,6))

plt.subplot(2,1,1)
plt.title('Closed-loop PID simulation')
plt.step(t,SP,'olive',label='SP', linewidth = 2, where='post')
plt.step(t,PV,'green',label=f"PV, gamma = {gamma_high}", linewidth = 2, where='post')
plt.step(t,PV_low_gamma,'purple',label=f"PV, gamma = {gamma_low}", linewidth = 2, where='post')
plt.ylabel('Value of SP, PV [°C]')
plt.legend(loc='upper right')
plt.xlim([0, TSim])

plt.subplot(2,1,2)
plt.step(t,MV,'darkgreen',label=f"MV, gamma = {gamma_high}", linewidth = 2,where='post')
plt.step(t,MV_low_gamma,'darkblue',label=f"MV, gamma = {gamma_low}", linewidth = 2,where='post')
plt.ylabel('Value of MV [%]')
plt.legend(loc='upper right')
plt.xlim([0, TSim])
plt.ylim([-5,105])

plt.xlabel('Time [s]')